<a href="https://colab.research.google.com/github/kidd89/streamlit-example/blob/master/singapore_resale_price.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [2]:
resaleflat_1 = pd.read_csv("/content/ResaleflatpricesbasedonregistrationdatefromJan2017onwards.csv")
#resaleflat_2 = pd.read_csv("/content/ResaleFlatPricesBasedonApprovalDate19901999.csv")
resaleflat_3 = pd.read_csv("/content/ResaleFlatPricesBasedonApprovalDate2000Feb2012.csv")
resaleflat_4 = pd.read_csv("/content/ResaleFlatPricesBasedonRegistrationDateFromJan2015toDec2016.csv")
resaleflat_5 = pd.read_csv("/content/ResaleFlatPricesBasedonRegistrationDateFromMar2012toDec2014.csv")

resaleflat = pd.concat((resaleflat_1,resaleflat_3,resaleflat_4,resaleflat_5), axis=0, sort=True)
df = resaleflat
df.head()
#df.shape

,block,flat_model,flat_type,floor_area_sqm,lease_commence_date,month,remaining_lease,resale_price,storey_range,street_name,town
0,406,Improved,2 ROOM,44.0,1979,2017-01,61 years 04 months,232000.0,10 TO 12,ANG MO KIO AVE 10,ANG MO KIO
1,108,New Generation,3 ROOM,67.0,1978,2017-01,60 years 07 months,250000.0,01 TO 03,ANG MO KIO AVE 4,ANG MO KIO
2,602,New Generation,3 ROOM,67.0,1980,2017-01,62 years 05 months,262000.0,01 TO 03,ANG MO KIO AVE 5,ANG MO KIO
3,465,New Generation,3 ROOM,68.0,1980,2017-01,62 years 01 month,265000.0,04 TO 06,ANG MO KIO AVE 10,ANG MO KIO
4,601,New Generation,3 ROOM,67.0,1980,2017-01,62 years 05 months,265000.0,01 TO 03,ANG MO KIO AVE 5,ANG MO KIO


In [3]:
df.describe()

,floor_area_sqm,lease_commence_date,resale_price
count,623931.000000,623931.000000,6.239310e+05
mean,96.779805,1990.353185,3.604173e+05
std,25.061414,11.282013,1.631463e+05
min,28.000000,1966.000000,2.800000e+04
25%,74.000000,1983.000000,2.400000e+05
50%,96.000000,1988.000000,3.370000e+05
75%,113.000000,1998.000000,4.480000e+05
max,297.000000,2022.000000,1.500000e+06


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 623931 entries, 0 to 52202
Data columns (total 11 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   block                623931 non-null  object 
 1   flat_model           623931 non-null  object 
 2   flat_type            623931 non-null  object 
 3   floor_area_sqm       623931 non-null  float64
 4   lease_commence_date  623931 non-null  int64  
 5   month                623931 non-null  object 
 6   remaining_lease      202077 non-null  object 
 7   resale_price         623931 non-null  float64
 8   storey_range         623931 non-null  object 
 9   street_name          623931 non-null  object 
 10  town                 623931 non-null  object 
dtypes: float64(2), int64(1), object(8)
memory usage: 57.1+ MB


In [5]:
df.isnull().sum() #has to fix remaning_lease bocz it has null points

block                       0
flat_model                  0
flat_type                   0
floor_area_sqm              0
lease_commence_date         0
month                       0
remaining_lease        421854
resale_price                0
storey_range                0
street_name                 0
town                        0
dtype: int64

In [6]:
df.head()

,block,flat_model,flat_type,floor_area_sqm,lease_commence_date,month,remaining_lease,resale_price,storey_range,street_name,town
0,406,Improved,2 ROOM,44.0,1979,2017-01,61 years 04 months,232000.0,10 TO 12,ANG MO KIO AVE 10,ANG MO KIO
1,108,New Generation,3 ROOM,67.0,1978,2017-01,60 years 07 months,250000.0,01 TO 03,ANG MO KIO AVE 4,ANG MO KIO
2,602,New Generation,3 ROOM,67.0,1980,2017-01,62 years 05 months,262000.0,01 TO 03,ANG MO KIO AVE 5,ANG MO KIO
3,465,New Generation,3 ROOM,68.0,1980,2017-01,62 years 01 month,265000.0,04 TO 06,ANG MO KIO AVE 10,ANG MO KIO
4,601,New Generation,3 ROOM,67.0,1980,2017-01,62 years 05 months,265000.0,01 TO 03,ANG MO KIO AVE 5,ANG MO KIO


##We had to convert categorical to numerical first to see correlations, now we redo the table using onehotencoding

In [7]:
#Convert first two char into integer for years left

df['remaining_lease_year'] = df['remaining_lease'].str.slice(stop=2).astype('float')
#Substring/slice the month out
df['remaining_lease_month'] = df['remaining_lease'].str.slice(start=9,stop=12)


replacemonth = {}
replacemonth[''] = 0
replacemonth['0 m'] = 0
df['remaining_lease_month'] = df['remaining_lease_month'].replace(replacemonth)


df['remaining_lease_month'] = df['remaining_lease_month'].astype('float') / 12

df['remaining_lease'] = df['remaining_lease_year'] + df['remaining_lease_month']
df = df.drop(['remaining_lease_year','remaining_lease_month'], axis=1)


df.month = df.month.apply(lambda x: float(x[0:4])+float(x[5:7])/12)
#Inpute missing values with remaining lease (take current year-month minus the lease start year)
df.remaining_lease[df.remaining_lease.isnull()] = df['month'] - df['lease_commence_date']

df.describe()


<ipython-input-7-982baa5cd6da>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.remaining_lease[df.remaining_lease.isnull()] = df['month'] - df['lease_commence_date']


,floor_area_sqm,lease_commence_date,month,remaining_lease,resale_price
count,623931.000000,623931.000000,623931.000000,623931.000000,6.239310e+05
mean,96.779805,1990.353185,2010.983314,33.802161,3.604173e+05
std,25.061414,11.282013,7.162800,26.912888,1.631463e+05
min,28.000000,1966.000000,2000.083333,-1.250000,2.800000e+04
25%,74.000000,1983.000000,2004.666667,13.833333,2.400000e+05
50%,96.000000,1988.000000,2010.083333,24.250000,3.370000e+05
75%,113.000000,1998.000000,2017.500000,53.833333,4.480000e+05
max,297.000000,2022.000000,2023.833333,97.750000,1.500000e+06


In [8]:
#Blocks in Singapore is rather irregular and does not impact on the pricing
#Lease commence date is highly correlated with remaining lease as remaining lease = current date - lease commence date

#Try using street name (high variation) instead of town , both are highly correlated so just use one

df = df.drop(columns=['block','lease_commence_date','town','flat_type'])

In [9]:
df.head()

,flat_model,floor_area_sqm,month,remaining_lease,resale_price,storey_range,street_name
0,Improved,44.0,2017.083333,61.333333,232000.0,10 TO 12,ANG MO KIO AVE 10
1,New Generation,67.0,2017.083333,60.583333,250000.0,01 TO 03,ANG MO KIO AVE 4
2,New Generation,67.0,2017.083333,62.416667,262000.0,01 TO 03,ANG MO KIO AVE 5
3,New Generation,68.0,2017.083333,62.083333,265000.0,04 TO 06,ANG MO KIO AVE 10
4,New Generation,67.0,2017.083333,62.416667,265000.0,01 TO 03,ANG MO KIO AVE 5


In [10]:
#Normalize the month, floor area and remaining lease (timeframe)
df.month = (df.month - df.month.mean()) / df.month.std()
df.floor_area_sqm = (df.floor_area_sqm - df.floor_area_sqm.mean()) / df.floor_area_sqm.std()
df.remaining_lease = (df.remaining_lease - df.remaining_lease.mean()) / df.remaining_lease.std()

In [11]:
df.head()

,flat_model,floor_area_sqm,month,remaining_lease,resale_price,storey_range,street_name
0,Improved,-2.106019,0.851625,1.022974,232000.0,10 TO 12,ANG MO KIO AVE 10
1,New Generation,-1.188273,0.851625,0.995106,250000.0,01 TO 03,ANG MO KIO AVE 4
2,New Generation,-1.188273,0.851625,1.063227,262000.0,01 TO 03,ANG MO KIO AVE 5
3,New Generation,-1.148371,0.851625,1.050841,265000.0,04 TO 06,ANG MO KIO AVE 10
4,New Generation,-1.188273,0.851625,1.063227,265000.0,01 TO 03,ANG MO KIO AVE 5


In [12]:
df.dtypes

flat_model          object
floor_area_sqm     float64
month              float64
remaining_lease    float64
resale_price       float64
storey_range        object
street_name         object
dtype: object

In [13]:
df.storey_range.unique()

array(['10 TO 12', '01 TO 03', '04 TO 06', '07 TO 09', '13 TO 15',
       '19 TO 21', '22 TO 24', '16 TO 18', '34 TO 36', '28 TO 30',
       '37 TO 39', '49 TO 51', '25 TO 27', '40 TO 42', '31 TO 33',
       '46 TO 48', '43 TO 45', '06 TO 10', '01 TO 05', '11 TO 15',
       '16 TO 20', '21 TO 25', '26 TO 30', '36 TO 40', '31 TO 35'],
      dtype=object)

In [14]:
#Get average of storey range e.g. 01 to 03 would be 2
df.storey_range = df.storey_range.apply(lambda x: (float(x[0:2]) + float(x[7:9]) ) /2 )

df.storey_range = (df.storey_range - df.storey_range.mean()) / df.storey_range.std()

In [15]:
df.head()

,flat_model,floor_area_sqm,month,remaining_lease,resale_price,storey_range,street_name
0,Improved,-2.106019,0.851625,1.022974,232000.0,-0.008688,ANG MO KIO AVE 10
1,New Generation,-1.188273,0.851625,0.995106,250000.0,-1.416997,ANG MO KIO AVE 4
2,New Generation,-1.188273,0.851625,1.063227,262000.0,-1.416997,ANG MO KIO AVE 5
3,New Generation,-1.148371,0.851625,1.050841,265000.0,-0.360766,ANG MO KIO AVE 10
4,New Generation,-1.188273,0.851625,1.063227,265000.0,-1.416997,ANG MO KIO AVE 5


In [16]:
storey_resale = df[["storey_range","resale_price"]]
storey_resale.corr()

#Storey range has correlation of 0.20 with resale price

,storey_range,resale_price
storey_range,1.000000,0.251316
resale_price,0.251316,1.000000


In [17]:
lease_resale = df[["remaining_lease","resale_price"]]
lease_resale.corr()

#remaining_lease has correlation of 0.46 with resale price

,remaining_lease,resale_price
remaining_lease,1.000000,0.451447
resale_price,0.451447,1.000000


In [18]:
month_resale_price = df[["month","resale_price"]]
month_resale_price.corr()

#month has correlation of 0.66 with resale price

,month,resale_price
month,1.000000,0.639518
resale_price,0.639518,1.000000


In [19]:
df.dtypes

flat_model          object
floor_area_sqm     float64
month              float64
remaining_lease    float64
resale_price       float64
storey_range       float64
street_name         object
dtype: object

In [20]:
df.head()

,flat_model,floor_area_sqm,month,remaining_lease,resale_price,storey_range,street_name
0,Improved,-2.106019,0.851625,1.022974,232000.0,-0.008688,ANG MO KIO AVE 10
1,New Generation,-1.188273,0.851625,0.995106,250000.0,-1.416997,ANG MO KIO AVE 4
2,New Generation,-1.188273,0.851625,1.063227,262000.0,-1.416997,ANG MO KIO AVE 5
3,New Generation,-1.148371,0.851625,1.050841,265000.0,-0.360766,ANG MO KIO AVE 10
4,New Generation,-1.188273,0.851625,1.063227,265000.0,-1.416997,ANG MO KIO AVE 5


In [21]:
all_flat_model = pd.get_dummies(df['flat_model'], prefix='flt')
all_streets = pd.get_dummies(df['street_name'], prefix='st')

In [22]:
df = df.drop(columns=['flat_model','street_name'])
df = pd.concat((df,all_flat_model,all_streets), axis=1)

In [23]:
df.head()

,floor_area_sqm,month,remaining_lease,resale_price,storey_range,flt_2-room,flt_3Gen,flt_Adjoined flat,flt_Apartment,flt_DBSS,...,st_YISHUN ST 72,st_YISHUN ST 81,st_YUAN CHING RD,st_YUNG AN RD,st_YUNG HO RD,st_YUNG KUANG RD,st_YUNG LOH RD,st_YUNG PING RD,st_YUNG SHENG RD,st_ZION RD
0,-2.106019,0.851625,1.022974,232000.0,-0.008688,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,-1.188273,0.851625,0.995106,250000.0,-1.416997,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,-1.188273,0.851625,1.063227,262000.0,-1.416997,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,-1.148371,0.851625,1.050841,265000.0,-0.360766,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,-1.188273,0.851625,1.063227,265000.0,-1.416997,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 623931 entries, 0 to 52202
Columns: 597 entries, floor_area_sqm to st_ZION RD
dtypes: float64(5), uint8(592)
memory usage: 380.8 MB


In [25]:
from sklearn.model_selection import train_test_split

In [26]:
#spliting

X = df.drop('resale_price', axis=1)
Y = df['resale_price']

print(X.shape)
print(Y.shape)

(623931, 596)
(623931,)


In [27]:
#Choose top 400 features for model out of 596

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

X = SelectKBest(f_classif, k=400).fit_transform(X, Y)

print(X.shape)

(623931, 400)


In [28]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [29]:
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(x_train,y_train)

LinearRegression()

In [30]:
pred_linReg = regressor.predict(x_test)
pred_linReg

array([451685.01241733, 507086.86916901, 262714.71082307, ...,
       114200.71047654, 228849.48043819, 510612.86992872])

In [31]:
y_test.head()

40243     455000.0
22153     580000.0
7471      330000.0
23521     395000.0
253491    259800.0
Name: resale_price, dtype: float64

In [32]:
(y_test - pred_linReg).head()

40243      3314.987583
22153     72913.130831
7471      67285.289177
23521     56266.813340
253491    20492.164982
Name: resale_price, dtype: float64

# lets see how our model performed

In [35]:
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [33]:
import math



In [36]:
math.sqrt(mean_squared_error(y_test, pred_linReg, multioutput='uniform_average'))

66763.83871814076

In [38]:
#Find RMSE for baseline model - i.e. take average house price

base = math.sqrt(((y_test - y_test.mean()) ** 2).mean())

base

#RMSE of $163250 for prediction of house price using average

163250.67109056178

In [39]:
#Get R^2 of prediction, 0.832

regressor.score(x_test,y_test)

0.8327474884092944

In [41]:
y_test.values

array([455000., 580000., 330000., ..., 132000., 353000., 475000.])

In [42]:
pred_linReg

array([451685.01241733, 507086.86916901, 262714.71082307, ...,
       114200.71047654, 228849.48043819, 510612.86992872])

In [47]:
result1 = np.concatenate(([y_test.values], [pred_linReg],[y_test.values - pred_linReg])).T

In [48]:
result1_df = pd.DataFrame(result1, columns=['actual','predict','difference'])

In [49]:
result1_df

,actual,predict,difference
0,455000.0,451685.012417,3314.987583
1,580000.0,507086.869169,72913.130831
2,330000.0,262714.710823,67285.289177
3,395000.0,338733.186660,56266.813340
4,259800.0,239307.835018,20492.164982
...,...,...,...
124782,308000.0,345380.330780,-37380.330780
124783,203000.0,286785.828922,-83785.828922
124784,132000.0,114200.710477,17799.289523
124785,353000.0,228849.480438,124150.519562


In [50]:
print(result1_df['difference'].mean())
print(result1_df['difference'].max())
print(result1_df['difference'].min())

-128.8795326127246
644785.2200749435
-503094.91225368297


In [53]:
print(regressor.intercept_) #.coef_ for all coefficients
#Base price is $339527

339527.1220470635
